# Question
The following figures show the Di Wang Tower in Shenzhen. The structure of Di Wang Tower is made up of steel and reinforced concrete (calculated as reinforced concrete), C terrain type, the design wind pressure $w_0 = 0.75 \text{ kN/m}^2$, period $T_1=6.38\text{ s}$, The first mode shape is listed in following table. To determine shape factor, the cross section could be considered as rectangle.

<center>
    <img src="./images/Diwang01.png" alt="The Elvation of the DiWang Tower" width="400"/>
    <img src="./images/Diwang02.png" alt="The Plan View of the DiWang Tower" width="400"/>
</center>

<br>

| $\dfrac{h}{H}$ | 0     | 0.1   | 0.2   | 0.3   | 0.4    | 0.5   | 0.6   | 0.7    | 0.8   | 0.9   | 1.0   |
| -------------- | ----- | ----- | ----- | ----- | ------ | ----- | ----- | ------ | ----- | ----- | ----- |
| Mode shape     | 0.0000| 0.0137| 0.0558| 0.1277| 0.2424| 0.3503| 0.4629| 0.5903| 0.7309| 0.8700| 1.0000| 
| Mass / ton     | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 1500  | 

Only consider the first mode, calculate **the extreme of top displacements** induced by the along-wind static and dynamic wind load in the given wind direction using the following methods:
1. Method recommended by the Chinese code for wind-resistant design of buildings (GB 50009-2012).
2. According to the quasi-steady assumption, convert the fluctuating wind velocity time histories in `windData` (**with a duration of 10 minutes and a sampling frequency of 10 Hz**) to wind load time histories $F_i(t)=[1/2\rho \bar{u_i}^2+\rho \bar{u_i} u'_i(t)] \cdot \mu_{s,i} \cdot A_i$, and then using stocastic vibration methods in time domain (such as the newmark-beta algorithm) and in frequency domain (such as the response spectrum analysis). Peak factor $g = 2.5$ should be used for the extreme value calculation, and the programming using frequency domain method is optional.

# Answer


In [3]:
import scipy.io as sio

# load data
simDataPath = './windData/windData.mat'
simData = sio.loadmat(simDataPath, squeeze_me=True, struct_as_record=False)
U = simData['U']
Z = simData['Z']
dt = simData['dt']
t = simData['t']
del simData
print("\nThe dimension of U is:", U.shape)


The dimension of U is: (11, 6000)


## 1.中国《建筑结构荷载规范》（GB 50009-2012）推荐的方法

In [43]:
import numpy as np

# 基本参数
H = 303.46  # 结构总高 (m)
w0 = 0.75  # 基本风压 (kN/m²)
T1 = 6.38  # 自振周期 (s)
mu_s = 1.4  # 体型系数（假设矩形截面）
g = 2.5  # 峰值因子
I_10 = 0.23  # C类地形湍流强度
k = 0.295  # C类地形参数
alpha1 = 0.261  # C类地形参数
k_w = 0.54  # C类地形脉动风参数
zeta1 = 0.05  # 阻尼比
B = 65.88  # 迎风面宽度 (m)
n_floors = 10  # 楼层数

h_H = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])  # 10个自由度对应的h/H
phi = np.array([0.0137, 0.0558, 0.1277, 0.2424, 0.3503, 
                0.4629, 0.5903, 0.7309, 0.8700, 1.0000])           # 振型系数（10个自由度）
m = np.array([2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 2500, 1500]) * 1000  # 质量 (kg)

# 计算风振系数beta_z
f1 = 1 / T1  # 频率 (Hz)
x1 = 30 * f1 / np.sqrt(k_w * w0)  
R = np.sqrt(np.pi / (6 * zeta1)) * (x1**2) / ((1 + x1**2) ** (4/3))

# 计算相关系数 rho_z 和 rho_x（结构整体参数）
rho_z = (10 * np.sqrt(H + 60 * np.exp(-H/60)) - 60) / H  # 公式(5.17)
rho_x = (10 * np.sqrt(B + 50 * np.exp(-B/50)) - 50) / B  # 公式(5.18)

# 逐层计算参数
beta_z_list = []
mu_z_list = []
F_list = []

for i in range(n_floors):  # 遍历h/H=0.1到1.0共10层（对应楼层1~10）
    # 当前层参数
    z_i = h_H[i] * H                 # 楼层高度 (m)
    phi_i = phi[i]                   # 振型系数
    mu_z_i = 0.544 * (z_i / 10)**0.44  # 风压高度变化系数
    
    # 计算B_z,i（公式5.16）
    B_z_i = k * (H**alpha1) * rho_z * rho_x * (phi_i / mu_z_i)
    
    # 计算beta_z,i
    beta_z_i = 1 + 2 * g * I_10 * B_z_i * np.sqrt(1 + R**2)
    
    # 记录参数
    beta_z_list.append(beta_z_i)
    mu_z_list.append(mu_z_i)
    
    # 计算单层风荷载
    A_i = (H / n_floors) * B          # 单层迎风面积 (m²)
    w_k_i = beta_z_i * mu_s * mu_z_i * w0_kN  # 风荷载标准值 (kN/m²)
    F_i = w_k_i * A_i                 # 单层静风荷载 (kN)
    F_list.append(F_i)

# 转换为数组
beta_z = np.array(beta_z_list)
mu_z = np.array(mu_z_list)
F = np.array(F_list)  # 各自由度静风荷载 (kN)

# --- 模态分析（严格按图片步骤）---
# 1. 计算模态质量 M1
M1 = np.sum(m * phi**2)  # 公式：M1 = Σ(m_i * φ_i²)

# 2. 计算模态力 F1
F1 = np.sum(phi * F * 1000)  # 公式：F1 = Σ(φ_i * F_i)，F单位转换为N

# 3. 计算模态坐标 η1
omega1 = 2 * np.pi / T1     # 圆频率 ω1 = 2πf
eta1 = F1 / (omega1**2 * M1)

# 4. 计算各自由度位移 X_i = η1 * φ_i
X = eta1 * phi  # 各自由度位移 (m)
X_top = X[-1]   # 顶部位移（最后一个自由度）

print("各自由度风振系数:",beta_z)
print("\n 风压高度变化系数:",mu_z)
print("\n 各自由度静风荷载 (kN)",F)
print(f"各自由度位移响应 (m):\n{np.round(X, 6)}")
print(f"顶部位移极值：{X_top:.6f} m")

各自由度风振系数: [1.00676665 1.02031582 1.03889653 1.06505482 1.08522099 1.1039331
 1.12384626 1.14459445 1.16342023 1.17933013]

 风压高度变化系数: [0.88659145 1.2027538  1.43766232 1.63166101 1.79999103 1.95033893
 2.08721255 2.21351839 2.33125748 2.44187562]

 各自由度静风荷载 (kN) [1873.68553405 2576.05836765 3135.25975695 3647.92785436 4100.46275012
 4519.57074367 4923.99865521 5318.37691721 5693.39343779 6045.09702907]
各自由度位移响应 (m):
[0.048469 0.197414 0.451787 0.857582 1.23932  1.637685 2.088411 2.585837
 3.077956 3.537881]
顶部位移极值：3.537881 m


## 2.时域方法（Newmark-β法）

In [58]:
import scipy.io as sio
import numpy as np

# === 1. 加载风数据 ===
simDataPath = 'D:/研一/风工程/2025homework-main/2025homework-main/Lecture10-WindInducedVibrationCalcutation/windData/windData.mat'
simData = sio.loadmat(simDataPath, squeeze_me=True, struct_as_record=False)
U = simData['U']         # 风速时程（m/s）
dt = simData['dt']       # 时间步长（s）
t = simData['t']         # 时间数组（s）
del simData
n_steps = len(t)         # 总时间步数

# === 2. 定义结构参数 ===
n_floors = 11            # 楼层数（对应h/H表格）
phi = np.array([0.0000, 0.0137, 0.0558, 0.1277, 0.2424, 0.3503, 
               0.4629, 0.5903, 0.7309, 0.8700, 1.0000])  # 振型

# 质量参数（转换为kg）
floor_masses = [2500*1000]*10 + [1500*1000]  
M1 = phi.T @ np.diag(floor_masses) @ phi  # 模态质量

# 频率参数
omega1 = 2 * np.pi / 6.38  # 圆频率
xi = 0.05                 # 阻尼比（假设5%）
C1 = 2 * xi * omega1 * M1  # 模态阻尼

# === 3. 计算风荷载时程F_i(t) ===
# 确保U的维度是 (n_steps, n_floors)
if U.shape[0] != n_steps:  # 如果时间步在第二维
    U = U.T                # 转置为 (n_steps, n_floors)

rho = 1.25                # 空气密度
mu_s = 1.3                # 体型系数
section_width = 65.88     # 截面宽度
floor_area = 303.46 /  n_floors * section_width  # 单层面积

F_t = np.zeros((n_steps, n_floors))
for i in range(n_floors):
    U_mean = np.mean(U[:, i])       # 取第i列（第i个楼层）的所有时间步
    u_fluct = U[:, i] - U_mean      # 脉动风速
    # 计算风荷载（确保所有项为向量）
    F_t[:, i] = 0.5 * rho * (U[:, i]**2) * mu_s * floor_area + \
                rho * U_mean * u_fluct * mu_s * floor_area
    
# === 4. Newmark-β法参数 ===
gamma = 0.5    # 纽马克参数
beta = 0.25    # 无条件稳定
F1_t = F_t @ phi  # 模态力时程（N）

# === 5. 初始化响应数组 ===
eta = np.zeros(n_steps)       # 模态位移
eta_dot = np.zeros(n_steps)   # 模态速度
eta_ddot = np.zeros(n_steps)  # 模态加速度

# 初始加速度（t=0时刻）
eta_ddot[0] = (F1_t[0] - C1*eta_dot[0] - (omega1**2 * M1)*eta[0]) / M1

# === 6. 时域积分循环 ===
for i in range(n_steps-1):
    # 预测位移和速度
    eta_pred = eta[i] + dt*eta_dot[i] + (0.5-beta)*dt**2*eta_ddot[i]
    eta_dot_pred = eta_dot[i] + (1-gamma)*dt*eta_ddot[i]
    
    # 计算残差力
    residual = F1_t[i+1] - (omega1**2 * M1)*eta_pred - C1*eta_dot_pred - M1*eta_ddot[i]
    
    # 有效刚度
    K_eff = M1 + gamma*dt*C1 + beta*dt**2*(omega1**2 * M1)
    
    # 更新加速度
    delta_eta_ddot = residual / K_eff
    eta_ddot[i+1] = eta_ddot[i] + delta_eta_ddot
    
    # 修正速度和位移
    eta_dot[i+1] = eta_dot_pred + gamma*dt*delta_eta_ddot
    eta[i+1] = eta_pred + beta*dt**2*delta_eta_ddot

# === 7. 计算物理位移 ===
X_t = np.outer(eta, phi)          # 各层位移时程
top_disp = X_t[:, -1]             # 顶部位移时程
X_max = 2.5 * np.std(top_disp)    # 峰值因子计算极值

print(f"Newmark-β法顶部位移极值: {X_max:.4f} m")

Newmark-β法顶部位移极值: 3.4651 m
